In [1]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style
alt.themes.enable('light')

ThemeRegistry.enable('light')

In [53]:
goods_df = pd.read_csv("goods.csv", skiprows=7)
goods_df.columns = ['date', 'value']

goods_df = goods_df[goods_df['date'].astype(str).str.contains("Q")]

def parse_quarters(date):
    #2023 Q1
    year, quarter = date.split(" ")
    quarter = quarter.replace("Q", "")
    month = (int(quarter) - 1) * 3 + 1
    return f"{year}-{month:02d}-01"

goods_df['date'] = goods_df['date'].apply(parse_quarters)

goods_df['series'] = 'Goods'


services_df = pd.read_csv("services.csv", skiprows=7)
services_df.columns = ['date', 'value']

services_df = services_df[services_df['date'].astype(str).str.contains("Q")]

services_df['date'] = services_df['date'].apply(parse_quarters)

services_df['series'] = 'Services'

df = pd.concat([goods_df, services_df])
df['value'] = df['value']/100
df

df = df.pivot(index='date', columns='series', values='value').reset_index()
df['Balance'] = df['Goods'] + df['Services']
df = df.melt(id_vars='date', value_vars=['Goods', 'Services', 'Balance'], var_name='series', value_name='value')



df['label'] = np.where(df['date'] == '2023-10-01', df.series, '')

base = alt.Chart(df).encode(
    x=alt.X('date:T', title=None),
    y=alt.Y('value:Q', title=None, axis=alt.Axis(format='%')),
    color=alt.Color('series:N', 
                    scale=alt.Scale(
                        domain=['Goods', 'Services', 'Balance'],
                        range=[              "#f4c245",
              "#36b7b4",
              "#122b39"
],
                    ),
                    title=None, legend=None)
)

bars = base.transform_filter("datum.series != 'Balance'").mark_area(interpolate='step-after').encode(
    y=alt.Y('value:Q', title=None),
    opacity=alt.value(1)
)

balance_line = base.transform_filter("datum.series == 'Balance'").mark_line(size=1.5).encode(
    y=alt.Y('value:Q', title=None),
    opacity=alt.value(1)
)


end_labels = base.mark_text(align='left', dx=5).encode(
    text='label'
)


chart = bars + end_labels  +  balance_line
chart = chart.properties(width=450, height=300)
chart


chart.save("trade_balance.json")
chart.save("trade_balance.png", scale_factor=3)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [ ]:

lines = base.mark_line()

end_labels = base.mark_text(align='left', dx=5).encode(
    text='label'
)

zero_line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(
    color='#676A86',
    strokeDash=[3,3]
    ).encode(
    y='y'
)

chart = (zero_line + lines + end_labels ).properties(
    width=500,
    height=400
)

chart


chart.save("trade_balance.json")
chart.save("trade_balance.png", scale_factor=3)

chart